In [68]:
import os
import requests
import pandas as pd
from dotenv import load_dotenv
import alpaca_trade_api as tradeapi
from pathlib import Path
import panel as pn
from panel.interact import interact
import plotly.express as px
pn.extension("plotly")

%matplotlib inline

In [69]:
# Load .env environment variables
load_dotenv()

True

In [70]:
# Set Alpaca API key and secret
alpaca_api_key = os.getenv("ALPACA_API_KEY")
alpaca_secret_key = os.getenv("ALPACA_SECRET_KEY")

In [71]:
# Create the Alpaca API object
alpaca = tradeapi.REST(
    alpaca_api_key,
    alpaca_secret_key,
    api_version="v2")

In [72]:
# Format start and end dates as ISO format for our bond indices
tickers = ['AGG', 'TIP', 'SHY', 'TLT', 'VNQ', 'XLE']
start_date = pd.Timestamp("1990-01-01", tz="America/New_York").isoformat()
end_date = pd.Timestamp("2022-04-20", tz="America/New_York").isoformat()

bond_index_portfolio = alpaca.get_bars(
    tickers,
    timeframe="1Day",
    start = start_date,
    end = end_date
).df

In [73]:
# sets the index as a dateformat
bond_index_portfolio.index = bond_index_portfolio.index.date
bond_index_portfolio.head()

,open,high,low,close,volume,trade_count,vwap,symbol
2015-12-01,108.54,108.8400,108.43,108.82,4259986,15562,108.754152,AGG
2015-12-02,108.73,108.7799,108.61,108.67,2462787,11581,108.684692,AGG
2015-12-03,108.41,108.4100,107.81,107.89,4634020,16801,108.040315,AGG
2015-12-04,108.05,108.3000,108.00,108.24,2182057,9796,108.192232,AGG
2015-12-07,108.30,108.5800,108.23,108.40,2143773,9104,108.460067,AGG


In [74]:
#reformats the data
bond_index_portfolio = bond_index_portfolio.drop(columns=['open','high','low','volume','trade_count','vwap'])
AGG_closes = bond_index_portfolio[bond_index_portfolio['symbol']=='AGG'].drop('symbol', axis=1)
TIP_closes = bond_index_portfolio[bond_index_portfolio['symbol']=='TIP'].drop('symbol', axis=1)
SHY_closes = bond_index_portfolio[bond_index_portfolio['symbol']=='SHY'].drop('symbol', axis=1)
TLT_closes = bond_index_portfolio[bond_index_portfolio['symbol']=='TLT'].drop('symbol', axis=1)
VNQ_closes = bond_index_portfolio[bond_index_portfolio['symbol']=='VNQ'].drop('symbol', axis=1)
XLE_closes = bond_index_portfolio[bond_index_portfolio['symbol']=='XLE'].drop('symbol', axis=1)
bond_index_portfolio_close = pd.concat([AGG_closes, TIP_closes, SHY_closes, TLT_closes,
VNQ_closes, XLE_closes],axis=1, keys=['AGG', 'TIP', 'SHY', 'TLT', 'VNQ', 'XLE'])

bond_index_all = pd.DataFrame()
bond_index_all["AGG"] = bond_index_portfolio_close["AGG"]["close"]
bond_index_all["TIP"] = bond_index_portfolio_close["TIP"]["close"]
bond_index_all["SHY"] = bond_index_portfolio_close["SHY"]["close"]
bond_index_all["TLT"] = bond_index_portfolio_close["TLT"]["close"]
bond_index_all["VNQ"] = bond_index_portfolio_close["VNQ"]["close"]
bond_index_all["XLE"] = bond_index_portfolio_close["XLE"]["close"]
bond_index_all.head()



,AGG,TIP,SHY,TLT,VNQ,XLE
2015-12-01,108.82,111.21,84.58,122.83,80.49,68.56
2015-12-02,108.67,110.83,84.51,122.87,78.85,66.44
2015-12-03,107.89,110.14,84.51,119.54,77.61,65.11
2015-12-04,108.24,110.42,84.53,120.58,79.08,64.74
2015-12-07,108.40,110.38,84.52,121.75,78.85,62.26


In [75]:
#csvpath = Path('C:/Users/15613/FinTechWork/FinTechProject1/BondIndexData.csv')
#bond_index_all.to_csv(csvpath)

In [76]:
#creates a selection widget
bond_tickers = pn.widgets.MultiChoice(name="Security List", value=['AGG'],
    options=tickers)
pn.Column(bond_tickers, height=200)

BokehModel(combine_events=True, render_bundle={'docs_json': {'868de7df-2a89-4b2d-ae89-c19d36884461': {'defs': …

Column(height=200)
    [0] MultiChoice(name='Security List', options=['AGG', 'TIP', ...], value=['AGG'])

In [78]:
#creates a new dataframe from the widget selection
selected_bond_indexes = bond_index_all[bond_tickers.value]
selected_bond_indexes

,AGG,SHY,TIP
2015-12-01,108.82,84.58,111.21
2015-12-02,108.67,84.51,110.83
2015-12-03,107.89,84.51,110.14
2015-12-04,108.24,84.53,110.42
2015-12-07,108.40,84.52,110.38
...,...,...,...
2022-04-13,104.82,83.33,121.18
2022-04-14,103.97,83.18,121.17
2022-04-18,103.65,83.18,121.13
2022-04-19,103.02,82.99,120.53
